In [ ]:
!pip install flyr
# !pip install exifread
# !pip install numpy
# !pip install flirimageextractor


In [5]:
import os
import csv
import json
import subprocess
import io
import concurrent.futures 
import flyr
from PIL import Image
from pyzbar.pyzbar import decode

# --- CONFIGURATION ---
try:
    base_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    base_dir = os.getcwd()

INPUT_FOLDER = os.path.join(base_dir, "flir e5 photodump")
OUTPUT_CSV = os.path.join(base_dir, "Thermal_Data_Log.csv")
EXIFTOOL_PATH = os.path.join(base_dir, "exiftool-12.35.exe") 

# Worker count: 10 is usually safe for file operations
MAX_WORKERS = 10

# --- HELPER FUNCTIONS ---

def get_process_flags():
    if os.name == 'nt':
        return subprocess.CREATE_NO_WINDOW
    return 0

def load_existing_records(csv_path):
    existing_signatures = set()
    if not os.path.exists(csv_path):
        return existing_signatures, False 
    try:
        with open(csv_path, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                ts = str(row.get("Timestamp", "")).strip()
                sn = str(row.get("Serial Number", "")).strip()
                if ts and sn:
                    existing_signatures.add((ts, sn))
        return existing_signatures, True
    except Exception:
        return existing_signatures, False

def get_folder_metadata_batch(folder_path, tool_path):
    print("Scanning metadata (Batch)...")
    tool_cmd = tool_path if os.path.exists(tool_path) else "exiftool"
    try:
        cmd = [
            tool_cmd, '-j', '-n', '-r', '-ext', 'jpg', '-ext', 'jpeg',
            '-DateTimeOriginal', '-CameraModel', '-CameraSerialNumber',
            '-SerialNumber', '-Emissivity', '-ObjectDistance',
            '-ImageDescription', '-UserComment',
            folder_path
        ]
        result = subprocess.run(cmd, capture_output=True, text=True, creationflags=get_process_flags())
        if result.stdout:
            return json.loads(result.stdout)
    except Exception as e:
        print(f"Error in metadata extraction: {e}")
    return []

# --- QR EXTRACTION HELPER ---
def extract_qr_with_exiftool(file_path):
    """
    Extracts the 'EmbeddedImage' (Real Photo) from the FLIR file 
    using ExifTool, then scans it for QR codes.
    """
    try:
        # Command to extract the binary image data
        cmd = [EXIFTOOL_PATH, "-b", "-EmbeddedImage", file_path]
        
        result = subprocess.run(
            cmd, 
            capture_output=True,
            creationflags=get_process_flags()
        )
        
        # If extraction worked, we have bytes
        if result.stdout:
            # Create a virtual image file in memory
            with Image.open(io.BytesIO(result.stdout)) as img:
                decoded = decode(img)
                if decoded:
                    # Return all found codes joined by pipe
                    return " | ".join([obj.data.decode("utf-8") for obj in decoded])
    except Exception:
        pass
        
    return "" # Return empty string if nothing found or error

# --- CORE PROCESSING WORKER ---
def process_single_image(meta):
    """
    Runs in a separate Thread. 
    """
    full_path = meta.get('SourceFile')
    filename = os.path.basename(full_path)
    
    # 1. Basic Metadata
    row_ts = str(meta.get("DateTimeOriginal", "Unknown"))
    row_model = str(meta.get("CameraModel", "Unknown"))
    row_sn = str(meta.get("CameraSerialNumber", ""))
    if not row_sn or row_sn == "None":
        row_sn = str(meta.get("SerialNumber", "Unknown"))
        
    # 2. Notes
    note = meta.get("ImageDescription")
    if not note:
        note = meta.get("UserComment")
    note_str = str(note).strip() if note else ""

    qr_str = ""
    status = "Success"
    
    # Thermal Data Placeholders
    center_t, max_t, min_t, avg_t, delta_t = "", "", "", "", ""

    try:
        # --- A. QR PROCESSING ---
        qr_str = extract_qr_with_exiftool(full_path)

        # --- B. THERMAL PROCESSING ---
        thermogram = flyr.unpack(full_path)
        thermal_data = thermogram.celsius
        
        min_val = thermal_data.min()
        max_val = thermal_data.max()
        avg_val = thermal_data.mean()
        delta_val = max_val - min_val
        h, w = thermal_data.shape
        center_val = thermal_data[h//2, w//2]

        center_t = f"{center_val:.1f}"
        max_t = f"{max_val:.1f}"
        min_t = f"{min_val:.1f}"
        avg_t = f"{avg_val:.1f}"
        delta_t = f"{delta_val:.1f}"

    except Exception as e:
        err_msg = str(e)
        if "not a FLIR" in err_msg or "Invalid" in err_msg:
            status = "Error: Not Radiometric"
        else:
            status = f"Error: {err_msg}"

    return [
        filename, row_ts, row_model, row_sn,
        center_t, max_t, min_t, avg_t, delta_t,
        meta.get("Emissivity", "N/A"),
        meta.get("ObjectDistance", "N/A"),
        note_str,
        qr_str,
        status
    ]

# --- MAIN EXECUTION ---
def main():
    if not os.path.exists(INPUT_FOLDER):
        print(f"Error: Input folder not found: {INPUT_FOLDER}")
        return

    # 1. Load DB
    existing_sigs, csv_exists = load_existing_records(OUTPUT_CSV)
    
    # 2. Batch Scan Metadata
    all_metadata = get_folder_metadata_batch(INPUT_FOLDER, EXIFTOOL_PATH)
    
    # 3. Filter New Files
    files_to_process = []
    for meta in all_metadata:
        if 'SourceFile' not in meta: continue
        ts = str(meta.get("DateTimeOriginal", "")).strip()
        sn = str(meta.get("CameraSerialNumber", "")).strip() or str(meta.get("SerialNumber", "")).strip()
        
        if (ts, sn) not in existing_sigs:
            files_to_process.append(meta)

    total_new = len(files_to_process)
    print(f"New images to process: {total_new}")
    if total_new == 0: return

    # 4. Threaded Processing
    headers = [
        "File Name", "Timestamp", "Camera Model", "Serial Number", 
        "Center Temp (C)", "Max Temp (C)", "Min Temp (C)", 
        "Avg Temp (C)", "Delta Temp (C)", 
        "Emissivity", "Distance", "Notes", "QR Data", "Status"
    ]

    with open(OUTPUT_CSV, mode='a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        if not csv_exists:
            writer.writerow(headers)

        print(f"Starting parallel processing with {MAX_WORKERS} threads...\n")
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            future_to_meta = {executor.submit(process_single_image, meta): meta for meta in files_to_process}
            
            for i, future in enumerate(concurrent.futures.as_completed(future_to_meta), 1):
                try:
                    row = future.result()
                    writer.writerow(row)
                    f.flush() 
                    
                    # --- UPDATED PROGRESS LINE ---
                    # end='\r' returns cursor to start of line
                    # " "*20 adds blank space to clear any previous longer text
                    filename = row[0]
                    print(f"Progress [{i}/{total_new}]: {filename}" + " "*30, end='\r', flush=True)
                    
                except Exception as exc:
                    print(f"\nError processing file: {exc}")

    print(f"\nDone! Processed {total_new} files.")

if __name__ == "__main__":
    main()

Scanning metadata (Batch)...
New images to process: 4
Starting parallel processing with 10 threads...

Progress [4/4]: FLIR0003 (1).jpg                              
Done! Processed 4 files.


In [8]:
import pandas as pd

# 1. Read the file
df = pd.read_csv("Thermal_Data_Log.csv")

# --- THE FIX ---
# ExifTool gives dates like "2025:12:01". Pandas prefers "2025-12-01".
# We use Regex to replace the first two colons with dashes.
df['Timestamp'] = df['Timestamp'].astype(str).str.replace(
    r'^(\d{4}):(\d{2}):(\d{2})', 
    r'\1-\2-\3', 
    regex=True
)

# 3. Convert to Datetime
# 'errors="coerce"' ensures that if one specific row is corrupt, 
# it turns into NaT (Not a Time) instead of crashing the whole script.
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

# 4. Sort (Oldest first)
df_sorted = df.sort_values(by='Timestamp', ascending=True)

df_sorted

,File Name,Timestamp,Camera Model,Serial Number,Center Temp (C),Max Temp (C),Min Temp (C),Avg Temp (C),Delta Temp (C),Emissivity,Distance,Notes,QR Data,Status
1,FLIR0003.jpg,2025-12-02 15:13:59.403000+02:00,FLIR E5 Pro,13333175,26.7,30.0,22.6,24.0,7.4,0.95,1,NaN,NaN,Success
3,FLIR0003 (1).jpg,2025-12-02 15:13:59.403000+02:00,FLIR E5 Pro,13333175,26.7,30.0,22.6,24.0,7.4,0.95,1,Camera 1,NaN,Success
0,FLIR0027.jpg,2025-12-02 15:14:03.709000+02:00,FLIR E5 Pro,13333134,34.0,34.2,22.7,27.0,11.5,0.95,1,NaN,NaN,Success
2,FLIR0027 (1).jpg,2025-12-02 15:14:03.709000+02:00,FLIR E5 Pro,13333134,34.0,34.2,22.7,27.0,11.5,0.95,1,Camera 2,NaN,Success


In [9]:
################################################ Ordered Notes Only by Timestamp ##########################################
import pandas as pd

# 1. Read the file
df = pd.read_csv("Thermal_Data_Log.csv")

# 2. Drop rows without notes
df_notes_only = df.dropna(subset=['Notes']).copy()

# --- THE FIX ---
# ExifTool gives dates like "2025:12:01". Pandas prefers "2025-12-01".
# We use Regex to replace the first two colons with dashes.
df_notes_only['Timestamp'] = df_notes_only['Timestamp'].astype(str).str.replace(
    r'^(\d{4}):(\d{2}):(\d{2})', 
    r'\1-\2-\3', 
    regex=True
)

# 3. Convert to Datetime
# 'errors="coerce"' ensures that if one specific row is corrupt, 
# it turns into NaT (Not a Time) instead of crashing the whole script.
df_notes_only['Timestamp'] = pd.to_datetime(df_notes_only['Timestamp'], errors='coerce')

# 4. Sort (Oldest first)
df_sorted = df_notes_only.sort_values(by='Timestamp', ascending=True)

df_sorted

,File Name,Timestamp,Camera Model,Serial Number,Center Temp (C),Max Temp (C),Min Temp (C),Avg Temp (C),Delta Temp (C),Emissivity,Distance,Notes,QR Data,Status
3,FLIR0003 (1).jpg,2025-12-02 15:13:59.403000+02:00,FLIR E5 Pro,13333175,26.7,30.0,22.6,24.0,7.4,0.95,1,Camera 1,NaN,Success
2,FLIR0027 (1).jpg,2025-12-02 15:14:03.709000+02:00,FLIR E5 Pro,13333134,34.0,34.2,22.7,27.0,11.5,0.95,1,Camera 2,NaN,Success
